Utilizaremos la libreria ***Python Surprise***, la cual permite crear sistemas de recomendación basados en filtrado colaborativo. Básicamente utilizaremos las calificaciones que han dado antes los usuarios para elaborar una predicción al respecto.

El paquete surprise no recibe directamente un objeto *DataFrame*. Tiene para parsear y leer un conjunto de datos a través de dos nuevos objetos: 

- Reader: Debemos especificar el valor mínimo y el valor máximo de las calificaciones ***'score'***
- Dataset: Nos permite leer los datos desde distintas fuentes

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
df_raitings = pd.read_csv("RatingsConsolidado.csv")
df_raitings.shape

(11024289, 5)

In [3]:
nueva_muestra = df_raitings.sample(n=400000)
nueva_muestra.shape

(400000, 5)

**CREACIÓN Y ENTRENAMIENTO**
*******************************************************************************

In [4]:
# Creamos los argumentos del reader 'min' y 'max' de la variable 'score'
reader = Reader(rating_scale=(nueva_muestra['score'].min(), nueva_muestra['score'].max()))

se crea una instancia de dataset usando la función ***"load_from_df"*** y le pasamos las variables del **df** y el objeto reader, que es el que se va a usar para poder operar sobre los puntajes

In [5]:
dataset = Dataset.load_from_df(nueva_muestra[['userId', 'title', 'score']], reader)

In [6]:
dataset

Podemos ver que ***dataset*** es una instancia de la clase **dataset** de la libreria ***surprise***<br> *surprise.dataset.DatasetAutoFolds*

In [7]:
#Separamos los parametros para 'train' y 'test'
trainset, testset = train_test_split(dataset, test_size=0.2)

In [8]:
# Usaremos un modelo de Singular Value De composition
param_grid = {'n_factors': 5, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.02}
model = SVD()

In [9]:
#Se entrena el modelo
model.fit(trainset)

In [10]:
# Predecimos
predictions = model.test(testset)

**EVALUACIÓN DEL MODELO**
***************************************************************************************

Estos valores de evaluación del modelo corresponden a dos métricas comunes utilizadas para evaluar el rendimiento de un modelo de regresión en Machine Learning: el RMSE (Root Mean Squared Error) y el MAE (Mean Absolute Error).

In [11]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.0281
MAE:  0.8121


0.8121429632565746

El RMSE mide la raíz cuadrada de la media de los errores cuadráticos entre las predicciones del modelo y los valores reales. En este caso, el valor de RMSE es 1.0015, lo que indica que, en promedio, las predicciones del modelo tienen un error de aproximadamente 1 unidad en la escala de la variable objetivo.

Por otro lado, el MAE mide la media de los errores absolutos entre las predicciones del modelo y los valores reales. En este caso, el valor de MAE es 0.7762, lo que indica que, en promedio, las predicciones del modelo tienen un error absoluto de aproximadamente 0.78 unidades en la escala de la variable objetivo.

In [12]:
#Optimización de hiperparametros para lograr el mejor resultado
from surprise.model_selection import cross_validate

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128,256]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, dataset, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0244  1.0247  1.0259  1.0250  0.0007  
Fit time          2.12    1.81    1.81    1.91    0.15    
Test time         1.06    0.82    0.84    0.90    0.11    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0297  1.0246  1.0215  1.0253  0.0034  
Fit time          1.85    1.83    1.85    1.84    0.01    
Test time         0.85    0.84    1.41    1.03    0.27    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0255  1.0224  1.0269  1.0249  0.0019  
Fit time          1.95    1.91    1.92    1.92    0.02    
Test time         0.85    0.88    0.87    0.87    0.01    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0278  1.0

A continuación se generará un segmento donde se probaran distintas combinaciones de hiperparámetros:

- **epochs:** Es la cantidad de pasadas sobre el dataset que hará el algoritmo empleando el descenso por el gradiente.
- **biased:** Usar parámetros de sesgo o no.
- **Ir_all:** Learning rate para todos los parámetros.
- **reg_all:** Término de regularización para todos los parámetros (lambda)

In [13]:

param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(dataset)

In [14]:
# Observamos performance del mejor modelo

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.0223435624073043
{'n_factors': 5, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.02}


In [15]:
from surprise import dump
import joblib

In [16]:
# Cargar el modelo desde el archivo
joblib.dump(model, 'modelo_entrenado.sav')


['modelo_entrenado.sav']

In [17]:
modelo_cargado = joblib.load('modelo_entrenado.sav')

def recomendacion(userId, id):


    prediction = model.predict(userId, id)

    if prediction.est >= 3.5:
        
        mensaje = "Esta selección es recomendada para ti, disfrutala!"
        
    else:
        
        mensaje = "Definitivamente debes ver otra peli XD"


    return mensaje

In [18]:
recomendacion(32300, 'as109')

'Esta selección es recomendada para ti, disfrutala!'

In [20]:
recomendacion(231, 'as123')

'Definitivamente debes ver otra peli XD'